In [ ]:
import matplotlib.pyplot as plt
import iris 
import cartopy
import cartopy.crs as ccrs
import netCDF4 as nc4

In [ ]:
infile='/data/users/frmk/CMIP6HighResMIP/u-ak306/ony/surface_T/nemo_ak306o_1y_19501201-19511201_grid-T.nc'
#infile ='/data/users/jme22rs/CMIP6HighResMIP/u-ak306/ony.nc.file/nemo_ak306o_1y_19501201_19511201_grid-T.nc'   
vmin = 0.0
vmax = 30.0
projection=ccrs.PlateCarree()

###----------------------------------------------------------------------------------

# Load in the data (using netCDF4)
#data=nc4.Dataset(infile)
# field = data.variables[varname][0]
# lat = data.variables['nav_lat'][:]
# lon = data.variables['nav_lon'][:]

# Load in the data (using iris)
cubes = iris.load(infile) 
# temperature = cubes[0], for example
field = cubes.variables['tos'][0]
lat = cubes.variables['nav_lat'][:]
lon = cubes.variables['nav_lon'][:]

# Set the figure size
plt.figure(figsize=(8, 5))

# Declare the projection of the plot
plt.gcf().add_subplot(111, projection=ccrs.PlateCarree())  

# Determine number of pixels in the subplot
bbox = plt.gca().get_window_extent().transformed(plt.gcf().dpi_scale_trans.inverted()) # Get axes bounding box in display space
nx = bbox.width * plt.gcf().get_dpi()     # Gives width in inches then converts to pixels
ny = bbox.height * plt.gcf().get_dpi()    # Gives height in inches then converts to pixels

# Reproject the data onto a regular grid (with dimensions set by the number of pixels in the subplot, as above)
x_extent = plt.gca().get_extent()[:2] # Return the first two items in the array
y_extent = plt.gca().get_extent()[-2:] # Return the last two items in the array
x, y = cartopy.img_transform.mesh_projection(projection, nx, ny, x_extents=x_extent, y_extents=y_extent)[:2] # Tell cartopy what coordinate system we are working in 
field = cartopy.img_transform.regrid(field, lon, lat, ccrs.PlateCarree(), projection, x, y)                  #

# Contour the data
c = plt.contourf(x, y, field, vmin=vmin, vmax=vmax)   # (lon,lat,data,colorbar minimum,colorbar maximum)

# Add coastlines, contour labels and a colour bar
plt.gca().coastlines()
plt.clabel(c, inline=False, colors='k')  # contour labels where the underlying contour is not removed, and is in black
plt.colorbar(c, orientation='horizontal', extend='both')  

plt.show()

